In [4]:
from urllib.request import urlretrieve
import os
import numpy as np
from sklearn.preprocessing import LabelBinarizer
#from tqdm import tqdm
from zipfile import ZipFile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

download('https://s3.amazonaws.com/video.udacity-data.com/topher/2016/December/584f6edd_data/data.zip','data.zip')
#download('https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/signnames.csv','signnames.csv')

print("All the files are downloaded")

def uncompress_features_labels(dir,name):
    if(os.path.isdir(name)):
        print('Data extracted')
    else:
        with ZipFile(dir) as zipf:
            zipf.extractall('data')
uncompress_features_labels('data.zip','data')
#uncompress_features_labels('data1.zip','data1')


def data_Files(mypath):
    onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
    print(onlyfiles)

#data_Files('datalab')
#data_Files('signnames.csv')
#data_Files('data1')
#!ls
print('Done')

Download file... data.zip ...
File downloaded
All the files are downloaded
Done


In [22]:
import os
import csv

from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Lambda, Cropping2D
from tqdm import tqdm
from keras.optimizers import Adam

samples = []
with open('./data/data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None) 
    for line in reader:
        samples.append(line)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples,test_size=0.15)
print(np.shape(train_samples))
print(train_samples[0])
print(np.shape(validation_samples))
#train_samples = len(train_samples)*6
#validation_samples = len(validation_samples)*6
from sklearn.utils import shuffle

import cv2
import numpy as np
import sklearn
import matplotlib.pyplot as plt

def generator(samples, batch_size=32):
    num_samples = len(samples)
   
    while 1: 
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                    for i in range(0,3):
                        name = './data/data/IMG/'+batch_sample[i].split('/')[-1]
                        
                        center_image = cv2.cvtColor(cv2.imread(name), cv2.COLOR_BGR2RGB)
                        center_angle = float(batch_sample[3])
                        images.append(center_image)
                        if(i==0):
                            angles.append(center_angle)
                        elif(i==1):
                            angles.append(center_angle+0.2)
                        elif(i==2):
                            angles.append(center_angle-0.2)
                        
                        images.append(cv2.flip(center_image,1))
                        if(i==0):
                            angles.append(center_angle*-1)
                        elif(i==1):
                            angles.append((center_angle+0.2)*-1)
                        elif(i==2):
                            angles.append((center_angle-0.2)*-1)
                            
                        
                    
               
            # trim image to only see section with road
        
            X_train = np.array(images)
            y_train = np.array(angles)
            #print(np.shape(X_train))
            #print(np.shape(y_train))
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

#print(np.shape(train_generator))
#print(np.shape(validation_generator))

ch, row, col = 3, 80, 320  # Trimmed image format






(6830, 7)
['IMG/center_2016_12_01_13_32_50_019.jpg', ' IMG/left_2016_12_01_13_32_50_019.jpg', ' IMG/right_2016_12_01_13_32_50_019.jpg', ' 0', ' 0.9855326', ' 0', ' 30.18692']
(1206, 7)


In [25]:
model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
#model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(3, 160, 320), output_shape=(3, 80, 320)))
model.add(Lambda(lambda x: (x / 127.5) - 1, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))

model.add(Convolution2D(24,5,5,subsample=(2,2)))
model.add(Activation('elu'))
model.add(Convolution2D(36,5,5,subsample=(2,2)))
model.add(Activation('elu'))
model.add(Convolution2D(48,5,5,subsample=(2,2)))
model.add(Activation('elu'))
model.add(Convolution2D(64,3,3))
model.add(Activation('elu'))
model.add(Convolution2D(64,3,3))
model.add(Activation('elu'))
model.add(Flatten())


model.add(Dense(100))

model.add(Activation('elu'))
model.add(Dense(50))
model.add(Activation('elu'))
model.add(Dense(10))
model.add(Activation('elu'))
model.add(Dense(1))

model.compile(loss='mse',optimizer='adam')
#model.fit(X_train,y_train,nb_epoch=5,validation_split=0.2,shuffle=True)

print(np.shape(train_samples))

#model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator, validation_steps=len(validation_samples), nb_epoch=3,verbose=1)



model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator,   nb_val_samples=len(validation_samples), nb_epoch=5, verbose=1)
model.save('model.h5')
print('Done')
model.summary()

(6830, 7)
Epoch 1/5
6720/6830 [============================>.] - ETA: 0s - loss: 0.0586

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


6912/6830 [==============================] - 13s - loss: 0.0580 - val_loss: 0.0271
Epoch 2/5
6912/6830 [==============================] - 12s - loss: 0.0276 - val_loss: 0.0182
Epoch 3/5
6912/6830 [==============================] - 12s - loss: 0.0251 - val_loss: 0.0214
Epoch 4/5
6912/6830 [==============================] - 12s - loss: 0.0263 - val_loss: 0.0224
Epoch 5/5
6996/6830 [==============================] - 13s - loss: 0.0227 - val_loss: 0.0169
Done
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_10 (Lambda)               (None, 160, 320, 3)   0           lambda_input_10[0][0]            
____________________________________________________________________________________________________
cropping2d_10 (Cropping2D)       (None, 65, 320, 3)    0           lambda_10[0][0]                  
___________________________________